In [ ]:
%%capture
!pip install kaleido
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load the data into a DataFrame
data = pd.read_csv("/content/drive/MyDrive/dataaaa/BN_Spot.csv")

In [ ]:
data['Date_time'] = pd.to_datetime(data['Date_time'])

# Set the 'Date_time' column as the index
data.set_index('Date_time', inplace=True)

# Remove rows with NaN values
data.dropna(inplace=True)

In [ ]:
# Resample the data in 15-minute intervals
resampled_data = data.resample('15T').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'})

In [ ]:
resampled_data.dropna(how='all', inplace=True)

In [ ]:
df = resampled_data

In [ ]:
# Reset the index and move it to a column
df.reset_index(inplace=True)

In [ ]:
from pandas.io.excel import read_excel
df1= read_excel("/content/drive/MyDrive/dataaaa/Report.xlsx")

In [ ]:
import pandas as pd
pd.options.display.max_rows = 9999


In [ ]:
import plotly.graph_objects as go
import zipfile

# Create a list to store the image filenames
image_filenames = []
# Convert the 'Date' column to datetime type
df['Date_time'] = pd.to_datetime(df['Date_time'], format='%Y-%m-%d %H:%M:%S')
for i in range(0,1304):
  buy_date_input = df1['Date_time'].iloc[i]
  sell_date_input = df1['Exit_datetime'].iloc[i]

  # Convert user input to datetime format
  buy_date = pd.to_datetime(buy_date_input)
  sell_date = pd.to_datetime(sell_date_input)

  start_date = buy_date
  end_date = sell_date 
  # # Select the specified date range
  filtered_data = df.loc[(df['Date_time'] >= start_date) & (df['Date_time'] <= end_date) & (df[['Open', 'High', 'Low', 'Close']].notnull().all(axis=1))]


  # Create the candlestick chart with filtered data
  fig = go.Figure(data=[go.Candlestick(x=filtered_data['Date_time'],
                                      open=filtered_data['Open'],
                                      high=filtered_data['High'],
                                      low=filtered_data['Low'],
                                      close=filtered_data['Close'],
                                      line_width=5)])
  # Mark the buying date on the chart with an arrow
  buy_data = filtered_data.loc[filtered_data['Date_time'] == buy_date]
  if not buy_data.empty:
  # Mark the buying date on the chart with an arrow
    fig.add_annotation(
        x=buy_date,
        y=filtered_data.loc[filtered_data['Date_time'] == buy_date]['Close'].iloc[0],
        text='Buying Date',
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor='darkgreen',
        ax=40,
        ay=50
    )

  # Mark the selling date on the chart with an arrow
  # Mark the selling date on the chart with an arrow
  sell_data = filtered_data.loc[filtered_data['Date_time'] == sell_date]
  if not sell_data.empty:

    fig.add_annotation(
        x=sell_date,
        y=filtered_data.loc[filtered_data['Date_time'] == sell_date]['Close'].iloc[0],
        text='Selling Date',
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor='darkred',
        ax=40,
        ay=-50
    )
  # Calculate profit/loss
  buy_data = filtered_data.loc[filtered_data['Date_time'] == buy_date]
  sell_data = filtered_data.loc[filtered_data['Date_time'] == sell_date]

  if not buy_data.empty and not sell_data.empty:
      buy_price = buy_data['Close'].iloc[0]
      sell_price = sell_data['Close'].iloc[0]
      profit_loss = sell_price - buy_price
  else:
      profit_loss = 0


  # Determine if it's a profit or loss at the buying position
  if profit_loss > 0:
      buy_annotation_text = f'Profit: {profit_loss} (Buy)'
      buy_annotation_color = 'darkgreen'
  elif profit_loss < 0:
      buy_annotation_text = f'Loss: {profit_loss} (Buy)'
      buy_annotation_color = 'darkred'
  else:
      buy_annotation_text = 'No profit or loss (Buy)'
      buy_annotation_color = 'black'

  # Determine if it's a profit or loss at the selling position
  if profit_loss > 0:
      sell_annotation_text = f'Profit: {profit_loss} (Sell)'
      sell_annotation_color = 'darkgreen'
  elif profit_loss < 0:
      sell_annotation_text = f'Loss: {profit_loss} (Sell)'
      sell_annotation_color = 'darkred'
  else:
      sell_annotation_text = 'No profit or loss (Sell)'
      sell_annotation_color = 'black'

  # Create a list to store the range breaks
  range_breaks = []

  # Iterate over the filtered_data['Date_time']
  # Create a new datetime value



  # Append the new value to the "Date_time" column
  for start, end in zip(filtered_data['Date_time'][:-1], filtered_data['Date_time'][1:]):
      # Calculate the time difference between consecutive dates
      time_diff = end - start
      
      # Check if the time difference is greater than the desired interval (e.g., 15 minutes)
      if time_diff > pd.Timedelta(minutes=15):
          # Add the range break for the time gap
          range_breaks.append(dict(bounds=[start, end]))
  # Add the range break for the last interval

  # # Remove the last range break if it is at the end
  #     if range_breaks and range_breaks[-1]['bounds'][1] == filtered_data['Date_time'].iloc[-1]:
  #         range_breaks.pop()

  # Add profit/loss information at the selling position to the chart
  sell_data = filtered_data.loc[filtered_data['Date_time'] == sell_date]

  if not sell_data.empty:
  # Add profit/loss information at the selling position to the chart
    fig.add_annotation(
        x=sell_date,
        y=filtered_data.loc[filtered_data['Date_time'] == sell_date]['Close'].iloc[0],
        text=sell_annotation_text,
        showarrow=False,
        font=dict(color=sell_annotation_color, size=12),
        align='center',
        xanchor='left',
        yanchor='top'
    )

  # Customize the layout

  fig.update_layout(
      title='Candlestick Chart for Specific Date Range',
      xaxis_title='Date',
      yaxis_title='Price',
      yaxis_tickprefix='$',
      showlegend=True,
        xaxis=dict(
            
            type='category',
            tickmode='array',
            range=[start_date, end_date],
            
            rangebreaks=range_breaks,
            ticktext=filtered_data['Date_time'].dt.strftime('%Y-%m-%d %H:%M'),  # Set the tick text as formatted dates
            tickvals=filtered_data['Date_time'].tolist(),
            constrain='domain',
            
              
            
                    ),
            margin=dict(l=50, r=50, t=50, b=50)  # Adjust the margin as needed
                    )
  image_filename = f"image_full{i}.png"
  fig.write_image(image_filename)
   # Add the image filename to the list
  image_filenames.append(image_filename)  
    # Add the image filename to the list
with zipfile.ZipFile("Amages7.zip", "w") as zipf:
    # Add each image file to the zip folder
    for filename in image_filenames:
        zipf.write(filename)

print("Images saved and zipped successfully. Filename: images.zip")
  
  # fig.write_image(f"image_full{i}.png")
  
  # Show the chart
  # fig.show()

In [ ]:
import zipfile

# Create a list to store the image filenames
image_filenames = []

for i in range(0,1304):
  buy_date_input = df1['Date_time'].iloc[i]

  # Specify the target date
  target_date = pd.to_datetime(buy_date_input)  # Replace with your desired target date

  # Find the index of the target date in the DataFrame
  target_index = df[df['Date_time'] == target_date].index
  if len(target_index) == 0:
        print(f"No data found for target date: {target_date}")
        continue
  target_index = target_index[0]      

  # Calculate the start index for the 11 candles after the target date
  start_index = max(0, target_index - 10)

  # Get the start date and end date based on the start index and target index
  start_date = df.loc[start_index, 'Date_time']
  end_date = df.loc[target_index + 11, 'Date_time']  # Add 11 candles (each candle is 15 minutes) to the target date

  # Filter the data based on the start date and end date
  filtered_data = df.loc[(df['Date_time'] >= start_date) & (df['Date_time'] <= end_date)]

  range_breaks = []
    # Iterate over the filtered_data['Date_time']
  for start, end in zip(filtered_data['Date_time'][:-1], filtered_data['Date_time'][1:]):
        # Calculate the time difference between consecutive dates
        time_diff = end - start
        
        # Check if the time difference is greater than a specified threshold (e.g., 1 hour)
        if time_diff > pd.Timedelta(minutes=15):
            # Add the range break for the non-trading hours or white spaces
            range_breaks.append(dict(bounds=[start, end]))


    # Create the candlestick chart with the filtered data
  fig = go.Figure(data=[go.Candlestick(x=filtered_data['Date_time'],
                                        open=filtered_data['Open'],
                                        high=filtered_data['High'],
                                        low=filtered_data['Low'],
                                        close=filtered_data['Close'],
                                        line_width=4)])

    # Customize the layout
  fig.update_layout(
        title='Candlestick Chart for 10 Candles Before Buy Date And After',
        xaxis_title='Date',
        yaxis_title='Price',
        yaxis_tickprefix='$',
        showlegend=True,
        margin=dict(l=0, r=0, t=30, b=0),  # Adjust the margin as needed
        xaxis=dict(
            type='category',
            tickmode='array',
            ticktext=filtered_data['Date_time'].dt.strftime('%Y-%m-%d %H:%M'),  # Set the tick text as formatted dates
            tickvals=filtered_data['Date_time'].tolist(),
            rangebreaks=range_breaks,
      )
  )
  # Add arrow and annotation for the buying date
  fig.add_annotation(
      x=target_date,
      y=filtered_data.loc[filtered_data['Date_time'] == target_date]['Close'].iloc[0],
      text='Buying Date',
      showarrow=True,
      arrowhead=2,
      arrowsize=1.5,
      arrowwidth=2,
      arrowcolor='green',
      ax=20,
      ay=-40,
      font=dict(color='green', size=12),
      align='center',
      xanchor='left',
      yanchor='top'
  )
  image_filename = f"image_buying{i}.png"
  fig.write_image(image_filename)
   # Add the image filename to the list
  image_filenames.append(image_filename)  
    # Add the image filename to the list
with zipfile.ZipFile("Amages5.zip", "w") as zipf:
    # Add each image file to the zip folder
  for filename in image_filenames:
     
      zipf.write(filename)

print("Images saved and zipped successfully. Filename: images.zip")
    # Show the chart
  # fig.show()

No data found for target date: 2015-04-01 12:45:00
No data found for target date: 2015-05-12 09:15:00
No data found for target date: 2015-08-11 09:15:00
No data found for target date: 2015-08-11 10:15:00
No data found for target date: 2015-08-11 10:30:00
No data found for target date: 2015-08-11 11:15:00
No data found for target date: 2015-10-01 09:15:00
No data found for target date: 2015-10-01 10:15:00
No data found for target date: 2015-10-05 09:15:00
No data found for target date: 2016-02-01 14:30:00
No data found for target date: 2016-03-01 09:15:00
No data found for target date: 2016-04-12 09:15:00
No data found for target date: 2016-05-09 10:00:00
No data found for target date: 2016-05-09 10:15:00
No data found for target date: 2016-05-09 10:45:00
No data found for target date: 2016-08-05 10:15:00
No data found for target date: 2016-08-05 10:45:00
No data found for target date: 2016-08-05 11:00:00
No data found for target date: 2016-08-10 10:00:00
No data found for target date: 

In [ ]:
import zipfile

# Create a list to store the image filenames
image_filenames = []

for i in range(0,1304):
  # Convert the 'Date' column to datetime type
  sell_date_input = df1['Exit_datetime'].iloc[i]

  # Specify the target date
  target_date = pd.to_datetime(sell_date_input)  # Replace with your desired target date

  # Find the index of the target date in the DataFrame
  target_index = df[df['Date_time'] == target_date].index
  if len(target_index) == 0:
      print(f"No data found for target date: {target_date}")
      continue
  target_index = target_index[0] 

  # Calculate the start index for the 11 candles before the target date
  start_index = max(0, target_index - 10)

  # Get the start date and end date based on the start index and target index
  start_date = df.loc[start_index, 'Date_time']
  end_date = df.loc[target_index + 11, 'Date_time']  # Add 11 candles (each candle is 15 minutes) to the target date

  # Filter the data based on the start date and end date
  filtered_data = df.loc[(df['Date_time'] >= start_date) & (df['Date_time'] <= end_date)]

  # Create a list to store the range breaks
  range_breaks = []

  # Iterate over the filtered_data['Date_time']
  for start, end in zip(filtered_data['Date_time'][:-1], filtered_data['Date_time'][1:]):
      # Calculate the time difference between consecutive dates
      time_diff = end - start
      
      # Check if the time difference is greater than a specified threshold (e.g., 15 minutes)
      if time_diff > pd.Timedelta(minutes=15):
          # Add the range break for the non-trading hours or white spaces
          range_breaks.append(dict(bounds=[start, end]))

  # Create the candlestick chart with the filtered data
  fig = go.Figure(data=[go.Candlestick(x=filtered_data['Date_time'],
                                      open=filtered_data['Open'],
                                      high=filtered_data['High'],
                                      low=filtered_data['Low'],
                                      close=filtered_data['Close'],
                                      line_width=4)])

  # Customize the layout
  fig.update_layout(
      title='Candlestick Chart for 10 Candles Before and After Sell Date',
      xaxis_title='Date',
      yaxis_title='Price',
      yaxis_tickprefix='$',
      showlegend=True,
      margin=dict(l=0, r=0, t=30, b=0),  # Adjust the margin as needed
      xaxis=dict(
          type='category',
          tickmode='array',
          ticktext=df['Date_time'].dt.strftime('%Y-%m-%d %H:%M'),  # Set the tick text as formatted dates
          tickvals=df['Date_time'].tolist(),
          rangebreaks=range_breaks,
      )
  )

  # Add arrow and annotation for the selling date
  fig.add_annotation(
      x=target_date,
      y=filtered_data.loc[filtered_data['Date_time'] == target_date]['Close'].iloc[0],
      text='Selling Date',
      showarrow=True,
      arrowhead=2,
      arrowsize=1.5,
      arrowwidth=2,
      arrowcolor='red',
      ax=20,
      ay=40,
      font=dict(color='red', size=12),
      align='center',
      xanchor='left',
      yanchor='bottom'
  )
  image_filename = f"image_selling{i}.png"
  fig.write_image(image_filename)
   # Add the image filename to the list
  image_filenames.append(image_filename)  
    # Add the image filename to the list
with zipfile.ZipFile("Amages4.zip", "w") as zipf:
    # Add each image file to the zip folder
  for filename in image_filenames:
     
      zipf.write(filename)

print("Images saved and zipped successfully. Filename: images.zip")
    # Show the chart